# AE simulations
Comparing results of E+ v9.2 to v9.5 where the incident solar angle bug was fixed.

Annual energy and demand data only.

3 vintages, 6 climate zones (NREL's default cities)

In [1]:
# Dependencies
import pandas as pd
import os
import numpy as np
import scipy.stats as sts
from scipy.stats import linregress

In [2]:
# # 2018 Egrid Costs (elec $/kWh, gas $/therm), change values per city
# gas = 0.83
# elec = 0.1314

---

## Import all csv files

In [3]:
energy92_path = "data/AllEnergy_92.csv"
energy95_path = "data/AllEnergy_95.csv"
demand92_path = "data/AllDemand_92.csv"
demand95_path = "data/AllDemand_95.csv"

In [4]:
energy92 = pd.read_csv(energy92_path)
energy95 = pd.read_csv(energy95_path)
demand92 = pd.read_csv(demand92_path)
demand95 = pd.read_csv(demand95_path)

In [5]:
energy92

,E+Version,Climate,System,Parametric,Heating [kBtu],Cooling [kBtu],Fan [kBtu],Heat [kWh],Cool [kWh],Fan [kWh],U,SHGC,Run
0,9.2,2A,Elec,HOU+elecres+slab+IECC_1990_para00001Table.csv,25220.00,14089.00,3307.00,7389.39,4127.99,968.94000,0.1,0.10,1
1,9.2,2A,Elec,HOU+elecres+slab+IECC_1990_para00002Table.csv,23643.00,16177.00,3798.00,6927.51,4739.98,1112.81000,0.1,0.25,2
2,9.2,2A,Elec,HOU+elecres+slab+IECC_1990_para00003Table.csv,22708.00,17683.00,4177.00,6653.42,5181.25,1223.82000,0.1,0.40,3
3,9.2,2A,Elec,HOU+elecres+slab+IECC_1990_para00004Table.csv,21711.00,19651.00,4656.00,6361.33,5757.74,1364.15000,0.1,0.55,4
4,9.2,2A,Elec,HOU+elecres+slab+IECC_1990_para00005Table.csv,20683.00,22213.00,5264.00,6060.18,6508.37,1542.50000,0.1,0.70,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1291,9.2,7,Gas,DLH+gasfurnace+slab+IECC_2018_para00032Table.csv,128161.87,1760.92,3516.04,37551.43,515.90,1030.19972,1.1,0.25,32
1292,9.2,7,Gas,DLH+gasfurnace+slab+IECC_2018_para00033Table.csv,122454.67,2589.41,3630.30,35879.22,758.70,1063.67790,1.1,0.40,33
1293,9.2,7,Gas,DLH+gasfurnace+slab+IECC_2018_para00034Table.csv,117201.25,3608.64,3870.18,34339.97,1057.30,1133.96274,1.1,0.55,34
1294,9.2,7,Gas,DLH+gasfurnace+slab+IECC_2018_para00035Table.csv,112694.09,4638.14,4532.28,33019.37,1359.00,1327.95804,1.1,0.70,35


In [6]:
energy95

,E+Version,Climate,System,Parametric,Heating [kBtu],Cooling [kBtu],Fan [kBtu],Heat [kWh],Cool [kWh],Fan [kWh],U,SHGC,Run
0,9.5,2A,Elec,HOU+elecres+slab+IECC_1990_para00001Table.csv,24971.00,14406.00,3368.00,7316.61,4221.03,986.70000,0.1,0.10,1
1,9.5,2A,Elec,HOU+elecres+slab+IECC_1990_para00002Table.csv,23449.00,16518.00,3867.00,6870.68,4839.71,1133.11000,0.1,0.25,2
2,9.5,2A,Elec,HOU+elecres+slab+IECC_1990_para00003Table.csv,22264.00,18593.00,4366.00,6523.36,5447.70,1279.16000,0.1,0.40,3
3,9.5,2A,Elec,HOU+elecres+slab+IECC_1990_para00004Table.csv,21280.00,20746.00,4889.00,6235.11,6078.66,1432.36000,0.1,0.55,4
4,9.5,2A,Elec,HOU+elecres+slab+IECC_1990_para00005Table.csv,20495.00,22843.00,5404.00,6005.09,6692.96,1583.38000,0.1,0.70,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1291,9.5,7,Gas,DLH+gasfurnace+slab+IECC_2018_para00032Table.csv,127914.13,1807.05,3518.20,37478.84,529.50,1030.83260,1.1,0.25,32
1292,9.5,7,Gas,DLH+gasfurnace+slab+IECC_2018_para00033Table.csv,121972.39,2699.13,3649.19,35737.91,790.80,1069.21267,1.1,0.40,33
1293,9.5,7,Gas,DLH+gasfurnace+slab+IECC_2018_para00034Table.csv,117283.16,3593.43,3854.46,34363.97,1052.90,1129.35678,1.1,0.55,34
1294,9.5,7,Gas,DLH+gasfurnace+slab+IECC_2018_para00035Table.csv,112717.21,4630.20,4517.80,33026.14,1356.60,1323.71540,1.1,0.70,35


In [7]:
demand92

,E+Version,Climate,System,Parametric,Heating [kBtuh],Cooling [kBtuh],Fan [kBtuh],Heat [kW],Cool [kW],Fan [kW],U,SHGC,Run
0,9.2,2A,Elec,HOU+elecres+slab+IECC_1990_para00001Table.csv,30.0,0.00,3.00,8.79,0.0,0.91000,0.1,0.10,1
1,9.2,2A,Elec,HOU+elecres+slab+IECC_1990_para00002Table.csv,30.0,0.00,1.00,8.79,0.0,0.42000,0.1,0.25,2
2,9.2,2A,Elec,HOU+elecres+slab+IECC_1990_para00003Table.csv,30.0,0.00,2.00,8.79,0.0,0.47000,0.1,0.40,3
3,9.2,2A,Elec,HOU+elecres+slab+IECC_1990_para00004Table.csv,30.0,0.00,2.00,8.79,0.0,0.52000,0.1,0.55,4
4,9.2,2A,Elec,HOU+elecres+slab+IECC_1990_para00005Table.csv,30.0,0.00,2.00,8.79,0.0,0.58000,0.1,0.70,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1291,9.2,7,Gas,DLH+gasfurnace+slab+IECC_2018_para00032Table.csv,0.0,5.03,1.24,0.00,1.5,0.36332,1.1,0.25,32
1292,9.2,7,Gas,DLH+gasfurnace+slab+IECC_2018_para00033Table.csv,0.0,5.72,1.45,0.00,1.7,0.42485,1.1,0.40,33
1293,9.2,7,Gas,DLH+gasfurnace+slab+IECC_2018_para00034Table.csv,0.0,6.23,1.61,0.00,1.8,0.47173,1.1,0.55,34
1294,9.2,7,Gas,DLH+gasfurnace+slab+IECC_2018_para00035Table.csv,0.0,7.10,1.87,0.00,2.1,0.54791,1.1,0.70,35


In [8]:
demand95

,E+Version,Climate,System,Parametric,Heating [kBtuh],Cooling [kBtuh],Fan [kBtuh],Heat [kW],Cool [kW],Fan [kW],U,SHGC,Run
0,9.5,2A,Elec,HOU+elecres+slab+IECC_1990_para00001Table.csv,30.0,0.00,3.00,8.79,0.0,0.91000,0.1,0.10,1
1,9.5,2A,Elec,HOU+elecres+slab+IECC_1990_para00002Table.csv,30.0,0.00,1.00,8.79,0.0,0.43000,0.1,0.25,2
2,9.5,2A,Elec,HOU+elecres+slab+IECC_1990_para00003Table.csv,30.0,0.00,2.00,8.79,0.0,0.48000,0.1,0.40,3
3,9.5,2A,Elec,HOU+elecres+slab+IECC_1990_para00004Table.csv,30.0,0.00,2.00,8.79,0.0,0.54000,0.1,0.55,4
4,9.5,2A,Elec,HOU+elecres+slab+IECC_1990_para00005Table.csv,30.0,0.00,2.00,8.79,0.0,0.59000,0.1,0.70,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1291,9.5,7,Gas,DLH+gasfurnace+slab+IECC_2018_para00032Table.csv,0.0,5.07,1.23,0.00,1.5,0.36039,1.1,0.25,32
1292,9.5,7,Gas,DLH+gasfurnace+slab+IECC_2018_para00033Table.csv,0.0,5.79,1.46,0.00,1.7,0.42778,1.1,0.40,33
1293,9.5,7,Gas,DLH+gasfurnace+slab+IECC_2018_para00034Table.csv,0.0,6.21,1.61,0.00,1.8,0.47173,1.1,0.55,34
1294,9.5,7,Gas,DLH+gasfurnace+slab+IECC_2018_para00035Table.csv,0.0,7.08,1.87,0.00,2.1,0.54791,1.1,0.70,35


-----

## E+ v 9.2 Slice into datasets

## 2A Houston 

In [9]:
energy92_hou1990_elec = energy92[0:36]

energy92_hou2006_elec = energy92[36:72]

energy92_hou2018_elec = energy92[72:108]

energy92_hou1990_gas = energy92[108:144]

energy92_hou2006_gas = energy92[144:180]

energy92_hou2018_gas = energy92[180:216]

demand92_hou1990_elec = demand92[0:36]

demand92_hou2006_elec = demand92[36:72]

demand92_hou2018_elec = demand92[72:108]

demand92_hou1990_gas = demand92[108:144]

demand92_hou2006_gas = demand92[144:180]

demand92_hou2018_gas = demand92[180:216]

In [10]:
#energy92_hou218_gas

## 3A Memphis

In [11]:
energy92_mem1990_elec = energy92[216:252]

energy92_mem2006_elec = energy92[252:288]

energy92_mem2018_elec = energy92[288:324]

energy92_mem1990_gas = energy92[324:360]

energy92_mem2006_gas = energy92[360:396]

energy92_mem2018_gas = energy92[396:432]

demand92_mem1990_elec = demand92[216:252]

demand92_mem2006_elec = demand92[252:288]

demand92_mem2018_elec = demand92[288:324]

demand92_mem1990_gas = demand92[324:360]

demand92_mem2006_gas = demand92[360:396]

demand92_mem2018_gas = demand92[396:432]

In [12]:
#energy92_mem1990_elec

## 4A Baltimore

In [13]:
energy92_bwi1990_elec = energy92[432:468]

energy92_bwi2006_elec = energy92[468:504]

energy92_bwi2018_elec = energy92[504:540]

energy92_bwi1990_gas = energy92[540:576]

energy92_bwi2006_gas = energy92[576:612]

energy92_bwi2018_gas = energy92[612:648]

demand92_bwi1990_elec = demand92[432:468]

demand92_bwi2006_elec = demand92[468:504]

demand92_bwi2018_elec = demand92[504:540]

demand92_bwi1990_gas = demand92[540:576]

demand92_bwi2006_gas = demand92[576:612]

demand92_bwi2018_gas = demand92[612:648]

In [14]:
#energy92_mem1990_elec

## 5A Chicago

In [15]:
energy92_ord1990_elec = energy92[648:684]

energy92_ord2006_elec = energy92[684:720]

energy92_ord2018_elec = energy92[720:756]

energy92_ord1990_gas = energy92[756:792]

energy92_ord2006_gas = energy92[792:828]

energy92_ord2018_gas = energy92[828:864]

demand92_ord1990_elec = demand92[648:684]

demand92_ord2006_elec = demand92[684:720]

demand92_ord2018_elec = demand92[720:756]

demand92_ord1990_gas = demand92[756:792]

demand92_ord2006_gas = demand92[792:828]

demand92_ord2018_gas = demand92[828:864]

In [16]:
#demand92_ord2018_gas

## 6A Burlington

In [17]:
energy92_bvt1990_elec = energy92[864:900]

energy92_bvt2006_elec = energy92[900:936]

energy92_bvt2018_elec = energy92[936:972]

energy92_bvt1990_gas = energy92[972:1008]

energy92_bvt2006_gas = energy92[1008:1044]

energy92_bvt2018_gas = energy92[1044:1080]

demand92_bvt1990_elec = demand92[864:900]

demand92_bvt2006_elec = demand92[900:936]

demand92_bvt2018_elec = demand92[936:972]

demand92_bvt1990_gas = demand92[972:1008]

demand92_bvt2006_gas = demand92[1008:1044]

demand92_bvt2018_gas = demand92[1044:1080]

In [18]:
#energy92_mem1990_elec

## 7 Duluth

In [19]:
energy92_dlh1990_elec = energy92[1080:1116]

energy92_dlh2006_elec = energy92[1116:1152]

energy92_dlh2018_elec = energy92[1152:1188]

energy92_dlh1990_gas = energy92[1188:1224]

energy92_dlh2006_gas = energy92[1224:1260]

energy92_dlh2018_gas = energy92[1260:1296]

demand92_dlh1990_elec = demand92[1080:1116]

demand92_dlh2006_elec = demand92[1116:1152]

demand92_dlh2018_elec = demand92[1152:1188]

demand92_dlh1990_gas = demand92[1188:1224]

demand92_dlh2006_gas = demand92[1224:1260]

demand92_dlh2018_gas = demand92[1260:1296]

In [20]:
#demand92_dlh2018_gas

-----

## E+ v 9.5 Slice into datasets

## 2A Houston 

In [21]:
energy95_hou1990_elec = energy95[0:36]

energy95_hou2006_elec = energy95[36:72]

energy95_hou2018_elec = energy95[72:108]

energy95_hou1990_gas = energy95[108:144]

energy95_hou2006_gas = energy95[144:180]

energy95_hou2018_gas = energy95[180:216]

demand95_hou1990_elec = demand95[0:36]

demand95_hou2006_elec = demand95[36:72]

demand95_hou2018_elec = demand95[72:108]

demand95_hou1990_gas = demand95[108:144]

demand95_hou2006_gas = demand95[144:180]

demand95_hou2018_gas = demand95[180:216]

## 3A Memphis

In [22]:
energy95_mem1990_elec = energy95[216:252]

energy95_mem2006_elec = energy95[252:288]

energy95_mem2018_elec = energy95[288:324]

energy95_mem1990_gas = energy95[324:360]

energy95_mem2006_gas = energy95[360:396]

energy95_mem2018_gas = energy95[396:432]

demand95_mem1990_elec = demand95[216:252]

demand95_mem2006_elec = demand95[252:288]

demand95_mem2018_elec = demand95[288:324]

demand95_mem1990_gas = demand95[324:360]

demand95_mem2006_gas = demand95[360:396]

demand95_mem2018_gas = demand95[396:432]

## 4A Baltimore

In [23]:
energy95_bwi1990_elec = energy95[432:468]

energy95_bwi2006_elec = energy95[468:504]

energy95_bwi2018_elec = energy95[504:540]

energy95_bwi1990_gas = energy95[540:576]

energy95_bwi2006_gas = energy95[576:612]

energy95_bwi2018_gas = energy95[612:648]

demand95_bwi1990_elec = demand95[432:468]

demand95_bwi2006_elec = demand95[468:504]

demand95_bwi2018_elec = demand95[504:540]

demand95_bwi1990_gas = demand95[540:576]

demand95_bwi2006_gas = demand95[576:612]

demand95_bwi2018_gas = demand95[612:648]

## 5A Chicago

In [24]:
energy95_ord1990_elec = energy95[648:684]

energy95_ord2006_elec = energy95[684:720]

energy95_ord2018_elec = energy95[720:756]

energy95_ord1990_gas = energy95[756:792]

energy95_ord2006_gas = energy95[792:828]

energy95_ord2018_gas = energy95[828:864]

demand95_ord1990_elec = demand95[648:684]

demand95_ord2006_elec = demand95[684:720]

demand95_ord2018_elec = demand95[720:756]

demand95_ord1990_gas = demand95[756:792]

demand95_ord2006_gas = demand95[792:828]

demand95_ord2018_gas = demand95[828:864]

## 6A Burlington

In [25]:
energy95_bvt1990_elec = energy95[864:900]

energy95_bvt2006_elec = energy95[900:936]

energy95_bvt2018_elec = energy95[936:972]

energy95_bvt1990_gas = energy95[972:1008]

energy95_bvt2006_gas = energy95[1008:1044]

energy95_bvt2018_gas = energy95[1044:1080]

demand95_bvt1990_elec = demand95[864:900]

demand95_bvt2006_elec = demand95[900:936]

demand95_bvt2018_elec = demand95[936:972]

demand95_bvt1990_gas = demand95[972:1008]

demand95_bvt2006_gas = demand95[1008:1044]


## 7 Duluth

In [26]:
energy95_dlh1990_elec = energy95[1080:1116]

energy95_dlh2006_elec = energy95[1116:1152]

energy95_dlh2018_elec = energy95[1152:1188]

energy95_dlh1990_gas = energy95[1188:1224]

energy95_dlh2006_gas = energy95[1224:1260]

energy95_dlh2018_gas = energy95[1260:1296]

demand95_dlh1990_elec = demand95[1080:1116]

demand95_dlh2006_elec = demand95[1116:1152]

demand95_dlh2018_elec = demand95[1152:1188]

demand95_dlh1990_gas = demand95[1188:1224]

demand95_dlh2006_gas = demand95[1224:1260]

demand95_dlh2018_gas = demand95[1260:1296]